In [ ]:
import argparse
import os, sys
import json
import numpy as np
import pandas as pd
import time

import WhiskiWrap as ww
from WhiskiWrap import load_whisker_data as lwd
# add directory above notebook to path
sys.path.append(os.path.dirname(os.path.abspath('')))
import whiskerpad as wp
# Check that whisk binaries are executables and update permissions if necessary
from wwutils.whisk_permissions import update_permissions
update_permissions()

from pathlib import Path
import cv2
import matplotlib.pyplot as plt

In [ ]:
splitUp=True
nproc=40
input_dir= Path('/home/wanglab/data/whisker_asym/wa001/test')
video_name='WA001_080224_01_TopCam_10s.mp4' 
base_name='WA001_080224_01'
input_file=input_dir/video_name

image_halves, image_side, face_side, fp = wp.get_side_image(str(input_file), splitUp)

# print each image halves side by side
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(image_halves[0], cmap='gray')
ax[0].axis('off')
ax[0].set_title(f"Face side: {face_side[0]}, Image side: {image_side[0]}")
ax[1].imshow(image_halves[1], cmap='gray')
ax[1].axis('off')
ax[1].set_title(f"Face side: {face_side[1]}, Image side: {image_side[1]}")
plt.show()

# # Save each side as tif
# output_dir = input_dir / f"frame_0_whiskers_vertical_{direction}"
# output_dir.mkdir(exist_ok=True)
# for i, side in enumerate(image_side):
#     output_file = output_dir / f'{base_name}_first_frame_{side}.tif'
#     cv2.imwrite(str(output_file), image_halves[i])

In [ ]:
# # Load whiskerpad json file
# whiskerpad_file = os.path.join(input_dir, f'whiskerpad_{base_name}.json')
# # whiskerpad_file = os.path.join(input_dir, f'whiskerpad_{os.path.basename(input_file).split(".")[0]}.json')

# if not os.path.exists(whiskerpad_file):
# # If whiskerpad file does not exist, create it
#     print('Creating whiskerpad parameters file.')
#     whiskerpad=wp.Params(input_file, splitUp, base_name)
#     # Get whiskerpad parameters
#     whiskerpadParams, splitUp = wp.WhiskerPad.get_whiskerpad_params(whiskerpad)
#     # Save whisking parameters to json file
#     wp.WhiskerPad.save_whiskerpad_params(whiskerpad, whiskerpadParams)

# with open(whiskerpad_file, 'r') as f:
#     whiskerpad_params = json.load(f)

# # Check that left and right whiskerpad parameters are defined
# if np.size(whiskerpad_params['whiskerpads'])<2:
#     raise Exception('Missing whiskerpad parameters in whiskerpad json file.')

# # Get side types (left / right or top / bottom)
# # side_types = [whiskerpad['FaceSide'].lower() for whiskerpad in whiskerpad_params['whiskerpads']]
# side_types = [whiskerpad['ImageSide'].lower() for whiskerpad in whiskerpad_params['whiskerpads']]

# side_types = face_side # rotated_face_side
# side_types = image_side

# # face image side defined by ImageBorderAxis !!
# print(f"Side types: {side_types}, direction: {direction}")

In [ ]:
# Comparing .measurements at different stages (only actually done for left side - right is all reclassified)
wmeas = {}
# WA001_080224_01_left_00000000_initial.measurements
# WA001_080224_01_left_00000000_classify.measurements
# WA001_080224_01_left_00000000_reclassify.measurements
output_dir=Path('/home/wanglab/data/whisker_asym/wa001/test/WT')
side_types=['left']
for im_side in side_types:
    print(f'Loading whisker measurements for {im_side} image side video')
    whisk_filename = output_dir / f'{base_name}_{im_side}_00000000_reclassify.whiskers'
    # Load whiskers
    wmeas[im_side] = ww.read_whisker_data(str(whisk_filename), 'df')
    # Keep only the first frame rows (frame_id=0) for each side
    # first_frame_indices = np.where(wmeas[im_side]['frame_id'] == 0)[0]
    # wmeas[im_side] = {key: value[first_frame_indices] for key, value in wmeas[im_side].items()}
    wmeas[im_side] = wmeas[im_side][wmeas[im_side]['frame_id'] == 0]

print(f"Whisker measurements for {len(wmeas)} sides loaded.")

In [ ]:
wmeas['left']
# wmeas_left = pd.DataFrame(wmeas['left'])
# wmeas_left

# "initial" is already very different from the original measurements done on first frame ...
# But at least length and pixel_length are close. That's not true for the classify measurements.

In [ ]:
# Load .whiskers and .measurements files separately for comparison
from WhiskiWrap import wfile_io
from WhiskiWrap.mfile_io import MeasurementsTable
output_dir=Path('/home/wanglab/data/whisker_asym/wa001/test/WT')
side_types=['left']
im_side='left'
whiskers_filename = str(output_dir / f'{base_name}_{im_side}_00000000_reclassify.whiskers')
whiskers = wfile_io.Load_Whiskers(whiskers_filename)
measurements_filename = whiskers_filename.replace('.whiskers', '.measurements')
M = MeasurementsTable(str(measurements_filename))
measurements = M.asarray()
# Keep only the first frame data
frame_id = 0
filtered_whiskers=whiskers[frame_id]
filtered_measurements = measurements[measurements[:, 1] == frame_id]

In [ ]:
# Create pandas dataframe from whiskers
columns = [
    "frame_id",
    "segment_id",
    "pixel_length"
]

# Initialize an empty list to store the data
data = []

# for frame, frame_whiskers in whiskers.items():
#     for whisker_id, wseg in frame_whiskers.items():
for whisker_id, wseg in filtered_whiskers.items():
    # Collect the data in a dictionary
    row = {
        "frame_id": wseg.time,
        "segment_id": wseg.id,
        "pixel_length": len(wseg.x)
    }
    # Append the dictionary to the list
    data.append(row)

# Convert the list of dictionaries to a DataFrame
whiskers_df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
whiskers_df

In [ ]:
# Convert measurements to a DataFrame
# Define the column names
columns = [
    "whisker_id",
    "frame_id",
    "segment_id",
    "length",
    "tracing_score",
    "angle_at_follicle",
    "mean_curvature",
    "follicle_position_x",
    "follicle_position_y",
    "tip_position_x",
    "tip_position_y"
]

measurements_df = pd.DataFrame(filtered_measurements, columns=columns)
# Convert the appropriate columns to integers
measurements_df[['whisker_id', 'frame_id', 'segment_id']] = measurements_df[['whisker_id', 'frame_id', 'segment_id']].astype(int)

# # Sort the DataFrame by whisker_id, treating -1 as a large positive value
# df['sort_key'] = df['whisker_id'].apply(lambda x: float('inf') if x == -1 else x)
# df = df.sort_values(by='sort_key').drop(columns='sort_key').reset_index(drop=True)

# Sort the DataFrame by length
# measurements_df = measurements_df.sort_values(by='length', ascending=False).reset_index(drop=True)

# Display the DataFrame by segment_id
measurements_df = measurements_df.sort_values(by='segment_id').reset_index(drop=True)

# Display the DataFrame
measurements_df


In [ ]:
# Ensure both DataFrames have the same index
whiskers_df.reset_index(drop=True, inplace=True)
measurements_df.reset_index(drop=True, inplace=True)

# Concatenate the DataFrames horizontally with custom suffixes for duplicate columns
combined_df = whiskers_df.join(measurements_df, lsuffix='_whisk', rsuffix='_meas')

# Display the combined DataFrame
combined_df